In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [2]:
# input
data_d_root <- '/oak/stanford/groups/mrivas/projects/biobank-methods-dev'
phe_f <- file.path(data_d_root, 'snpnet-elastic-net/phenotype.phe')
PRS_d <- file.path(data_d_root, 'snpnet-clumping', 'train')
covar_score_d <- file.path(data_d_root, 'snpnet-PRScs/covar_betas_train_val')

# constants
covars <- c('age', 'sex', paste0('PC', 1:10))

# output
out_f <- 'snpnet-clumping.train.eval.tsv'


In [3]:
read_BETAs <- function(beta_f){
    fread(beta_f)
}


In [4]:
read_PRS <- function(sscore_f){
    fread(
        cmd=paste('zstdcat', sscore_f),
        select=c('#FID', 'IID', 'SCORE1_SUM'),
        colClasses=c('#FID'='character', 'IID'='character')
    ) %>%
    rename('FID'='#FID', 'geno_score'='SCORE1_SUM')
}


In [5]:
read_covar_score <- function(covar_score_f){
    fread(
        cmd=paste('zstdcat', covar_score_f),
        select=c('#FID', 'IID', 'Estimate'),
        colClasses=c('#FID'='character', 'IID'='character')
    ) %>%
    rename('FID'='#FID', 'covar_score'='Estimate')
    
}


In [6]:
perform_eval <- function(response, pred, metric.type){
    if(metric.type == 'r2'){
        summary(lm(response ~ 1 + pred))$r.squared
    }else{
#         pROC::auc(pROC::roc(response, pred))        
        pred.obj <- ROCR::prediction(pred, factor(response - 1))
        auc.obj <- ROCR::performance(pred.obj, measure = 'auc')
        auc.obj@y.values[[1]]
    }
}


In [7]:
phe_df <- fread(phe_f, colClasses=c('FID'='character', 'IID'='character')) %>%
mutate(ID = paste(FID, IID, sep='_')) %>%
column_to_rownames('ID')


In [8]:
eval_line_build <- function(score_test_df, phe, metric.type, split_string){
    data.frame(
        phe        = phe,
        split      = split_string,
        geno       = perform_eval(
            score_test_df$phe,
            score_test_df$geno_score,
            metric.type
        ),
        covar      = perform_eval(
            score_test_df$phe,
            score_test_df$covar_score,
            metric.type
        ),
        geno_covar = perform_eval(
            score_test_df$phe,
            score_test_df$geno_covar_score,
            metric.type
        ),
        stringsAsFactors = F
    )    
}


In [9]:
eval_df <- c('INI50', 'INI21001', 'HC269', 'HC382') %>%
lapply(function(phe){   
    c('1e-5', '1e-4', '1e-3') %>%
    lapply(function(p_thr){
        
        metric.type <- ifelse(str_replace_all(phe, '[0-9]', '') %in% c('INI', 'QT_FC'), 'r2', 'auc')

        df <- phe_df %>% 
        select(all_of(c('FID', 'IID', phe, 'split'))) %>%
        rename(!!'phe' := all_of(phe)) %>%
        left_join(
            read_PRS(file.path(PRS_d, sprintf('%s.p1_%s.sscore.zst', phe, p_thr))),
            by=c("FID", "IID")
        ) %>%
        left_join(
            read_covar_score(file.path(covar_score_d, sprintf('%s.covar.scores.tsv', phe))), 
            by=c("FID", "IID")
        ) %>%
        mutate(geno_covar_score = geno_score + covar_score) %>%
        drop_na(phe) %>%
        filter(phe != -9)

        nvars <- read_BETAs(
            file.path(PRS_d, sprintf('%s.p1_%s.clumped.plink.tsv', phe, p_thr))
        ) %>% nrow()
        
        bind_rows(
            df %>%
            filter(split == 'train')%>%
            eval_line_build(phe, metric.type, 'train'),

            df %>%
            filter(split == 'val')%>%
            eval_line_build(phe, metric.type, 'val'),

            df %>%
            filter(split == 'test')%>%
            eval_line_build(phe, metric.type, 'test')
        ) %>%
        mutate(
            p1_thr = p_thr,
            n_variables = nvars 
        )
    }) %>% bind_rows()        
}) %>% bind_rows() %>%
select(phe, p1_thr, split, geno, covar, geno_covar, n_variables)


In [10]:
eval_df

phe,p1_thr,split,geno,covar,geno_covar,n_variables
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
INI50,1e-5,train,0.07338269,0.534437096,0.27789433,4919
INI50,1e-5,val,0.05772298,0.530465444,0.25334154,4919
INI50,1e-5,test,0.05607385,0.533574168,0.25071727,4919
INI50,1e-4,train,0.08755674,0.534437096,0.28109526,7548
INI50,1e-4,val,0.06606014,0.530465444,0.24946666,7548
INI50,1e-4,test,0.06398967,0.533574168,0.24589476,7548
INI50,1e-3,train,0.11196420,0.534437096,0.29534082,13264
INI50,1e-3,val,0.07705061,0.530465444,0.24786860,13264
INI50,1e-3,test,0.07376580,0.533574168,0.24299273,13264


In [11]:
eval_df %>%
fwrite(out_f, sep='\t', na = "NA", quote=F)


In [12]:
eval_df %>%
filter(split == 'val')

phe,p1_thr,split,geno,covar,geno_covar,n_variables
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<int>
INI50,1e-5,val,0.05772298,0.53046544,0.25334154,4919
INI50,1e-4,val,0.06606014,0.53046544,0.24946666,7548
INI50,1e-3,val,0.07705061,0.53046544,0.24786860,13264
INI21001,1e-5,val,0.02425495,0.01036046,0.02846933,945
INI21001,1e-4,val,0.02816167,0.01036046,0.03149800,2043
INI21001,1e-3,val,0.03741326,0.01036046,0.04015869,5523
HC269,1e-5,val,0.59899757,0.69592252,0.68259964,244
HC269,1e-4,val,0.60038899,0.69592252,0.67605458,500
HC269,1e-3,val,0.59877126,0.69592252,0.66093522,1607
